# Bootstrap Methods for Financial Time Series

This notebook demonstrates the bootstrap methods available in the MFE Toolbox for financial time series analysis. Bootstrap techniques are essential for statistical inference with dependent data, where standard independence assumptions are often violated.

## Overview

Bootstrap methods are resampling techniques that allow for statistical inference without making strong distributional assumptions. In financial econometrics, where data often exhibits serial dependence, specialized bootstrap methods are required to preserve the dependence structure.

The MFE Toolbox provides several bootstrap methods specifically designed for dependent data:

1. **Block Bootstrap**: Resamples blocks of consecutive observations to preserve short-range dependence
2. **Stationary Bootstrap**: Uses random block lengths for improved stationarity properties
3. **Model Confidence Set (MCS)**: Identifies the set of models that are statistically indistinguishable from the best model
4. **Bootstrap Reality Check and SPA Test**: Tests for superior predictive ability among competing forecasting models

All bootstrap implementations in the MFE Toolbox leverage NumPy for efficient array operations and Numba for performance acceleration of computationally intensive resampling algorithms.

## Setup and Imports

Let's start by importing the necessary modules from the MFE Toolbox and other required libraries.

In [ ]:
# Standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, List, Tuple, Optional, Union, Any, Callable
import asyncio
import time
from datetime import datetime, timedelta

# MFE Toolbox imports
import mfe
from mfe.models.bootstrap import BlockBootstrap, StationaryBootstrap, ModelConfidenceSet, BSDS
from mfe.models.univariate import GARCH
from mfe.models.distributions import Normal, StudentT
from mfe.utils.data_transformations import returns_from_prices

# Configure plotting
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_context("notebook", font_scale=1.2)
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['figure.dpi'] = 100

# Display version information
print(f"MFE Toolbox version: {mfe.__version__}")

## 1. Generating Example Data

Let's generate some example time series data with known properties to demonstrate bootstrap methods. We'll create an AR(1) process with some volatility clustering.

In [ ]:
# Set random seed for reproducibility
np.random.seed(42)

# Generate an AR(1) process with volatility clustering
def generate_ar1_garch11_process(n: int = 1000, ar_coef: float = 0.7, 
                                omega: float = 0.05, alpha: float = 0.1, beta: float = 0.85) -> np.ndarray:
    """
    Generate an AR(1) process with GARCH(1,1) innovations.
    
    Parameters
    ----------
    n : int
        Number of observations
    ar_coef : float
        AR(1) coefficient
    omega : float
        GARCH constant term
    alpha : float
        ARCH parameter
    beta : float
        GARCH parameter
        
    Returns
    -------
    np.ndarray
        Generated time series
    """
    # Initialize arrays
    y = np.zeros(n)
    sigma2 = np.zeros(n)
    sigma2[0] = omega / (1 - alpha - beta)  # Unconditional variance
    
    # Generate innovations
    z = np.random.normal(0, 1, n)
    
    # Generate process
    for t in range(1, n):
        # GARCH variance
        sigma2[t] = omega + alpha * (z[t-1]**2 * sigma2[t-1]) + beta * sigma2[t-1]
        
        # AR(1) process with GARCH innovations
        y[t] = ar_coef * y[t-1] + np.sqrt(sigma2[t]) * z[t]
    
    return y

# Generate data
n_obs = 1000
ar_data = generate_ar1_garch11_process(n=n_obs)

# Create a date range for our time series
dates = pd.date_range(start='2020-01-01', periods=n_obs, freq='B')

# Create a Pandas Series with DatetimeIndex
ts_data = pd.Series(ar_data, index=dates, name='value')

# Plot the time series
plt.figure(figsize=(14, 6))
plt.plot(ts_data.index, ts_data.values)
plt.title('Simulated AR(1) Process with GARCH(1,1) Innovations')
plt.xlabel('Date')
plt.ylabel('Value')
plt.grid(True)
plt.tight_layout()
plt.show()

# Display summary statistics
print("Time Series Summary Statistics:")
print(ts_data.describe())

Let's also check the autocorrelation structure of our simulated data to confirm it has the expected properties.

In [ ]:
# Check autocorrelation structure
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# ACF of time series
plot_acf(ts_data.values, lags=30, ax=axes[0], title='Autocorrelation Function (ACF)')
axes[0].set_xlabel('Lag')
axes[0].set_ylabel('Autocorrelation')

# PACF of time series
plot_pacf(ts_data.values, lags=30, ax=axes[1], title='Partial Autocorrelation Function (PACF)')
axes[1].set_xlabel('Lag')
axes[1].set_ylabel('Partial Autocorrelation')

plt.tight_layout()
plt.show()

# Check for volatility clustering (autocorrelation in squared returns)
plt.figure(figsize=(14, 6))
plot_acf(ts_data.values**2, lags=30, title='ACF of Squared Values (Volatility Clustering)')
plt.xlabel('Lag')
plt.ylabel('Autocorrelation')
plt.tight_layout()
plt.show()

## 2. Block Bootstrap

The block bootstrap method resamples blocks of consecutive observations to preserve the dependence structure in the data. This is particularly useful for time series data where observations are not independent.

In [ ]:
# Create a block bootstrap with block size 50
block_size = 50
block_bootstrap = BlockBootstrap(block_size=block_size)

# Generate 1000 bootstrap samples
num_bootstrap_samples = 1000
bootstrap_samples = block_bootstrap.generate(ts_data.values, num_samples=num_bootstrap_samples)

# Compute bootstrap statistics (e.g., mean of each sample)
bootstrap_means = np.array([sample.mean() for sample in bootstrap_samples])
bootstrap_stds = np.array([sample.std() for sample in bootstrap_samples])

# Plot the bootstrap distribution of the mean
plt.figure(figsize=(14, 6))
sns.histplot(bootstrap_means, kde=True, stat='density', bins=50)
plt.axvline(ts_data.mean(), color='r', linestyle='--', label='Sample Mean')
plt.title('Block Bootstrap Distribution of Mean')
plt.xlabel('Mean')
plt.ylabel('Density')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Compute bootstrap confidence interval for the mean
mean_ci = np.percentile(bootstrap_means, [2.5, 97.5])
print(f"95% Bootstrap Confidence Interval for Mean: [{mean_ci[0]:.6f}, {mean_ci[1]:.6f}]")
print(f"Sample Mean: {ts_data.mean():.6f}")

# Plot the bootstrap distribution of the standard deviation
plt.figure(figsize=(14, 6))
sns.histplot(bootstrap_stds, kde=True, stat='density', bins=50)
plt.axvline(ts_data.std(), color='r', linestyle='--', label='Sample Std Dev')
plt.title('Block Bootstrap Distribution of Standard Deviation')
plt.xlabel('Standard Deviation')
plt.ylabel('Density')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Compute bootstrap confidence interval for the standard deviation
std_ci = np.percentile(bootstrap_stds, [2.5, 97.5])
print(f"95% Bootstrap Confidence Interval for Std Dev: [{std_ci[0]:.6f}, {std_ci[1]:.6f}]")
print(f"Sample Std Dev: {ts_data.std():.6f}")

### Visualizing Bootstrap Samples

Let's visualize a few bootstrap samples to understand how the block bootstrap works.

In [ ]:
# Plot a few bootstrap samples
plt.figure(figsize=(14, 10))

# Plot original data
plt.subplot(3, 1, 1)
plt.plot(ts_data.values, label='Original Data')
plt.title('Original Time Series')
plt.ylabel('Value')
plt.grid(True)
plt.legend()

# Plot two bootstrap samples
plt.subplot(3, 1, 2)
plt.plot(bootstrap_samples[0], label='Bootstrap Sample 1')
plt.title(f'Block Bootstrap Sample 1 (Block Size = {block_size})')
plt.ylabel('Value')
plt.grid(True)
plt.legend()

plt.subplot(3, 1, 3)
plt.plot(bootstrap_samples[1], label='Bootstrap Sample 2')
plt.title(f'Block Bootstrap Sample 2 (Block Size = {block_size})')
plt.xlabel('Time')
plt.ylabel('Value')
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()

### Block Size Selection

The choice of block size is crucial for the block bootstrap. Let's examine how different block sizes affect the bootstrap distribution.

In [ ]:
# Try different block sizes
block_sizes = [10, 30, 50, 100]
bootstrap_means_by_block = {}
bootstrap_stds_by_block = {}

for bs in block_sizes:
    # Create bootstrap with current block size
    bootstrap = BlockBootstrap(block_size=bs)
    
    # Generate bootstrap samples
    samples = bootstrap.generate(ts_data.values, num_samples=1000)
    
    # Compute statistics
    bootstrap_means_by_block[bs] = np.array([sample.mean() for sample in samples])
    bootstrap_stds_by_block[bs] = np.array([sample.std() for sample in samples])

# Plot bootstrap distributions for different block sizes
plt.figure(figsize=(14, 10))

# Plot distributions of means
plt.subplot(2, 1, 1)
for bs in block_sizes:
    sns.kdeplot(bootstrap_means_by_block[bs], label=f'Block Size = {bs}')
plt.axvline(ts_data.mean(), color='r', linestyle='--', label='Sample Mean')
plt.title('Block Bootstrap Distribution of Mean for Different Block Sizes')
plt.xlabel('Mean')
plt.ylabel('Density')
plt.legend()
plt.grid(True)

# Plot distributions of standard deviations
plt.subplot(2, 1, 2)
for bs in block_sizes:
    sns.kdeplot(bootstrap_stds_by_block[bs], label=f'Block Size = {bs}')
plt.axvline(ts_data.std(), color='r', linestyle='--', label='Sample Std Dev')
plt.title('Block Bootstrap Distribution of Standard Deviation for Different Block Sizes')
plt.xlabel('Standard Deviation')
plt.ylabel('Density')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

# Compute confidence intervals for different block sizes
print("95% Bootstrap Confidence Intervals for Different Block Sizes:")
print("
Mean:")
for bs in block_sizes:
    ci = np.percentile(bootstrap_means_by_block[bs], [2.5, 97.5])
    width = ci[1] - ci[0]
    print(f"Block Size = {bs}: [{ci[0]:.6f}, {ci[1]:.6f}], Width = {width:.6f}")

print("
Standard Deviation:")
for bs in block_sizes:
    ci = np.percentile(bootstrap_stds_by_block[bs], [2.5, 97.5])
    width = ci[1] - ci[0]
    print(f"Block Size = {bs}: [{ci[0]:.6f}, {ci[1]:.6f}], Width = {width:.6f}")

### Numba-Accelerated Implementation

The block bootstrap implementation in the MFE Toolbox uses Numba's just-in-time compilation for performance-critical operations. Let's compare the performance of the Numba-accelerated implementation with a pure Python implementation.

In [ ]:
# Define a pure Python implementation of block bootstrap
def generate_block_bootstrap_indices_python(n: int, block_size: int, seed: Optional[int] = None) -> np.ndarray:
    """
    Generate bootstrap indices without Numba acceleration.
    
    Parameters
    ----------
    n : int
        Length of the original series
    block_size : int
        Size of each block
    seed : int, optional
        Random seed
        
    Returns
    -------
    np.ndarray
        Array of bootstrap indices
    """
    # Set random seed if provided
    if seed is not None:
        np.random.seed(seed)
    
    # Initialize indices array
    indices = np.zeros(n, dtype=np.int64)
    
    # Generate blocks until we have enough indices
    pos = 0
    while pos < n:
        # Randomly select block start
        block_start = np.random.randint(0, n - block_size + 1)
        
        # Add block indices
        block_end = min(pos + block_size, n)
        indices[pos:block_end] = np.arange(block_start, block_start + (block_end - pos))
        
        # Move position
        pos = block_end
    
    return indices

def generate_block_bootstrap_python(data: np.ndarray, num_samples: int, block_size: int) -> List[np.ndarray]:
    """
    Generate block bootstrap samples using pure Python implementation.
    
    Parameters
    ----------
    data : np.ndarray
        Original data array
    num_samples : int
        Number of bootstrap samples to generate
    block_size : int
        Size of each block
        
    Returns
    -------
    List[np.ndarray]
        List of bootstrap samples
    """
    n = len(data)
    samples = []
    
    for _ in range(num_samples):
        # Generate indices
        indices = generate_block_bootstrap_indices_python(n, block_size)
        
        # Create bootstrap sample
        sample = data[indices]
        samples.append(sample)
    
    return samples

# Compare performance
block_size = 50
num_samples = 100

# Time Numba-accelerated version
start_time = time.time()
block_bootstrap = BlockBootstrap(block_size=block_size)
numba_samples = block_bootstrap.generate(ts_data.values, num_samples=num_samples)
numba_time = time.time() - start_time

# Time Python version
start_time = time.time()
python_samples = generate_block_bootstrap_python(ts_data.values, num_samples, block_size)
python_time = time.time() - start_time

print(f"Numba-accelerated version: {numba_time:.4f} seconds")
print(f"Python version: {python_time:.4f} seconds")
print(f"Speedup factor: {python_time / numba_time:.1f}x")

## 3. Stationary Bootstrap

The stationary bootstrap improves upon the block bootstrap by using random block lengths, which enhances the stationarity properties of the resampled series.

In [ ]:
# Create a stationary bootstrap with expected block size 50
expected_block_size = 50
stationary_bootstrap = StationaryBootstrap(expected_block_size=expected_block_size)

# Generate 1000 bootstrap samples
num_bootstrap_samples = 1000
stat_bootstrap_samples = stationary_bootstrap.generate(ts_data.values, num_samples=num_bootstrap_samples)

# Compute bootstrap statistics
stat_bootstrap_means = np.array([sample.mean() for sample in stat_bootstrap_samples])
stat_bootstrap_stds = np.array([sample.std() for sample in stat_bootstrap_samples])

# Plot the bootstrap distribution of the mean
plt.figure(figsize=(14, 6))
sns.histplot(stat_bootstrap_means, kde=True, stat='density', bins=50)
plt.axvline(ts_data.mean(), color='r', linestyle='--', label='Sample Mean')
plt.title('Stationary Bootstrap Distribution of Mean')
plt.xlabel('Mean')
plt.ylabel('Density')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Compute bootstrap confidence interval for the mean
stat_mean_ci = np.percentile(stat_bootstrap_means, [2.5, 97.5])
print(f"95% Bootstrap Confidence Interval for Mean: [{stat_mean_ci[0]:.6f}, {stat_mean_ci[1]:.6f}]")
print(f"Sample Mean: {ts_data.mean():.6f}")

# Plot the bootstrap distribution of the standard deviation
plt.figure(figsize=(14, 6))
sns.histplot(stat_bootstrap_stds, kde=True, stat='density', bins=50)
plt.axvline(ts_data.std(), color='r', linestyle='--', label='Sample Std Dev')
plt.title('Stationary Bootstrap Distribution of Standard Deviation')
plt.xlabel('Standard Deviation')
plt.ylabel('Density')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Compute bootstrap confidence interval for the standard deviation
stat_std_ci = np.percentile(stat_bootstrap_stds, [2.5, 97.5])
print(f"95% Bootstrap Confidence Interval for Std Dev: [{stat_std_ci[0]:.6f}, {stat_std_ci[1]:.6f}]")
print(f"Sample Std Dev: {ts_data.std():.6f}")

### Comparing Block and Stationary Bootstrap

Let's compare the distributions of statistics from the block bootstrap and stationary bootstrap.

In [ ]:
# Compare distributions of means
plt.figure(figsize=(14, 6))
sns.kdeplot(bootstrap_means, label='Block Bootstrap')
sns.kdeplot(stat_bootstrap_means, label='Stationary Bootstrap')
plt.axvline(ts_data.mean(), color='r', linestyle='--', label='Sample Mean')
plt.title('Comparison of Bootstrap Distributions of Mean')
plt.xlabel('Mean')
plt.ylabel('Density')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Compare distributions of standard deviations
plt.figure(figsize=(14, 6))
sns.kdeplot(bootstrap_stds, label='Block Bootstrap')
sns.kdeplot(stat_bootstrap_stds, label='Stationary Bootstrap')
plt.axvline(ts_data.std(), color='r', linestyle='--', label='Sample Std Dev')
plt.title('Comparison of Bootstrap Distributions of Standard Deviation')
plt.xlabel('Standard Deviation')
plt.ylabel('Density')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Compare confidence intervals
print("95% Bootstrap Confidence Intervals:")
print(f"Block Bootstrap Mean: [{mean_ci[0]:.6f}, {mean_ci[1]:.6f}], Width = {mean_ci[1] - mean_ci[0]:.6f}")
print(f"Stationary Bootstrap Mean: [{stat_mean_ci[0]:.6f}, {stat_mean_ci[1]:.6f}], Width = {stat_mean_ci[1] - stat_mean_ci[0]:.6f}")
print(f"
Block Bootstrap Std Dev: [{std_ci[0]:.6f}, {std_ci[1]:.6f}], Width = {std_ci[1] - std_ci[0]:.6f}")
print(f"Stationary Bootstrap Std Dev: [{stat_std_ci[0]:.6f}, {stat_std_ci[1]:.6f}], Width = {stat_std_ci[1] - stat_std_ci[0]:.6f}")

### Asynchronous Processing with Progress Tracking

For large-scale bootstrap operations, the MFE Toolbox provides asynchronous processing with progress tracking.

In [ ]:
# Define a progress callback function
def progress_callback(percent: float, message: str) -> None:
    """
    Callback function to report progress.
    
    Parameters
    ----------
    percent : float
        Percentage of completion (0-100)
    message : str
        Progress message
    """
    print(f"{percent:.1f}% complete: {message}")

# Define an asynchronous function to run bootstrap
async def run_bootstrap_async() -> Tuple[float, float]:
    """
    Run bootstrap asynchronously with progress tracking.
    
    Returns
    -------
    Tuple[float, float]
        Lower and upper bounds of confidence interval
    """
    # Create a stationary bootstrap
    bootstrap = StationaryBootstrap(expected_block_size=50)
    
    # Generate 5,000 bootstrap samples asynchronously with progress tracking
    print("Generating 5,000 bootstrap samples asynchronously...")
    bootstrap_samples = await bootstrap.generate_async(
        ts_data.values, 
        num_samples=5000,
        progress_callback=progress_callback
    )
    
    # Compute bootstrap statistics
    bootstrap_means = np.array([sample.mean() for sample in bootstrap_samples])
    conf_interval = np.percentile(bootstrap_means, [2.5, 97.5])
    
    return conf_interval[0], conf_interval[1]

# Run the async function
lower_bound, upper_bound = await run_bootstrap_async()
print(f"
95% Bootstrap Confidence Interval: [{lower_bound:.6f}, {upper_bound:.6f}]")
print(f"Sample Mean: {ts_data.mean():.6f}")

## 4. Bootstrap for Parameter Uncertainty in GARCH Models

Let's use bootstrap methods to assess parameter uncertainty in GARCH models. We'll first estimate a GARCH model, then use bootstrap to construct confidence intervals for the parameters.

In [ ]:
# Generate financial returns data with GARCH effects
def generate_garch_returns(n: int = 1000, 
                          omega: float = 0.05, 
                          alpha: float = 0.1, 
                          beta: float = 0.85) -> np.ndarray:
    """
    Generate returns from a GARCH(1,1) process.
    
    Parameters
    ----------
    n : int
        Number of observations
    omega : float
        GARCH constant term
    alpha : float
        ARCH parameter
    beta : float
        GARCH parameter
        
    Returns
    -------
    np.ndarray
        Generated returns
    """
    # Initialize arrays
    returns = np.zeros(n)
    sigma2 = np.zeros(n)
    sigma2[0] = omega / (1 - alpha - beta)  # Unconditional variance
    
    # Generate innovations
    z = np.random.normal(0, 1, n)
    
    # Generate process
    for t in range(1, n):
        # GARCH variance
        sigma2[t] = omega + alpha * returns[t-1]**2 + beta * sigma2[t-1]
        
        # Returns
        returns[t] = np.sqrt(sigma2[t]) * z[t]
    
    return returns

# Generate returns data
np.random.seed(42)
n_obs = 1000
true_omega = 0.05
true_alpha = 0.1
true_beta = 0.85
returns_data = generate_garch_returns(n=n_obs, omega=true_omega, alpha=true_alpha, beta=true_beta)

# Create a date range for our returns
dates = pd.date_range(start='2020-01-01', periods=n_obs, freq='B')

# Create a Pandas Series with DatetimeIndex
returns_series = pd.Series(returns_data, index=dates, name='returns')

# Plot the returns
plt.figure(figsize=(14, 6))
plt.plot(returns_series.index, returns_series.values)
plt.title('Simulated GARCH(1,1) Returns')
plt.xlabel('Date')
plt.ylabel('Returns')
plt.grid(True)
plt.tight_layout()
plt.show()

# Estimate GARCH(1,1) model
garch_model = GARCH(p=1, q=1, distribution=Normal())
garch_results = garch_model.fit(returns_data)

# Display estimation results
print("GARCH(1,1) Estimation Results:")
print(f"Log-Likelihood: {garch_results.log_likelihood:.4f}")
print(f"AIC: {garch_results.aic:.4f}")
print(f"BIC: {garch_results.bic:.4f}")
print("
Parameter Estimates:")
for name, value, std_err, t_stat, p_value in zip(
    garch_results.parameter_names,
    garch_results.parameters,
    garch_results.std_errors,
    garch_results.t_stats,
    garch_results.p_values
):
    print(f"{name}: {value:.6f} (SE: {std_err:.6f}, t: {t_stat:.4f}, p: {p_value:.4f})")

# Compare with true parameters
print("
True Parameters:")
print(f"omega: {true_omega:.6f}")
print(f"alpha: {true_alpha:.6f}")
print(f"beta: {true_beta:.6f}")

### Parametric Bootstrap for GARCH Models

Now let's use parametric bootstrap to assess parameter uncertainty. We'll simulate data from the estimated model, re-estimate the model on each simulated dataset, and construct confidence intervals for the parameters.

In [ ]:
# Parametric bootstrap for GARCH models
def parametric_bootstrap_garch(returns: np.ndarray, model: GARCH, 
                              estimated_params: np.ndarray, 
                              num_bootstrap: int = 1000) -> Dict[str, np.ndarray]:
    """
    Perform parametric bootstrap for GARCH models.
    
    Parameters
    ----------
    returns : np.ndarray
        Original returns data
    model : GARCH
        Estimated GARCH model
    estimated_params : np.ndarray
        Estimated parameters
    num_bootstrap : int, optional
        Number of bootstrap replications
        
    Returns
    -------
    Dict[str, np.ndarray]
        Dictionary of bootstrap parameter distributions
    """
    # Get parameter names
    param_names = model.parameter_names
    
    # Initialize arrays to store bootstrap parameters
    bootstrap_params = {name: np.zeros(num_bootstrap) for name in param_names}
    
    # Perform bootstrap replications
    for i in range(num_bootstrap):
        # Progress update
        if (i + 1) % 100 == 0 or i == 0:
            print(f"Bootstrap replication {i + 1}/{num_bootstrap}")
        
        # Simulate data from the estimated model
        sim_result = model.simulate(
            estimated_params,
            len(returns),
            n_simulations=1,
            initial_value=returns[0],
            initial_variance=np.var(returns)
        )
        sim_returns = sim_result.returns[:, 0]  # Get the first (and only) simulation
        
        try:
            # Re-estimate the model on simulated data
            bootstrap_result = model.fit(sim_returns)
            
            # Store parameters
            for j, name in enumerate(param_names):
                bootstrap_params[name][i] = bootstrap_result.parameters[j]
        except Exception as e:
            # If estimation fails, use original parameters
            print(f"Warning: Estimation failed for bootstrap sample {i + 1}: {e}")
            for j, name in enumerate(param_names):
                bootstrap_params[name][i] = estimated_params[j]
    
    return bootstrap_params

# Run parametric bootstrap (with fewer replications for demonstration)
num_bootstrap = 200  # Use a smaller number for demonstration
bootstrap_params = parametric_bootstrap_garch(
    returns_data, garch_model, garch_results.parameters, num_bootstrap=num_bootstrap
)

In [ ]:
# Plot bootstrap distributions of parameters
param_names = garch_model.parameter_names
fig, axes = plt.subplots(len(param_names), 1, figsize=(14, 4 * len(param_names)))

for i, name in enumerate(param_names):
    # Get true parameter value if available
    true_value = None
    if name == 'omega':
        true_value = true_omega
    elif name == 'alpha[1]':
        true_value = true_alpha
    elif name == 'beta[1]':
        true_value = true_beta
    
    # Plot bootstrap distribution
    sns.histplot(bootstrap_params[name], kde=True, ax=axes[i])
    
    # Add vertical lines for estimated and true values
    axes[i].axvline(garch_results.parameters[i], color='r', linestyle='--', 
                   label=f'Estimated: {garch_results.parameters[i]:.6f}')
    
    if true_value is not None:
        axes[i].axvline(true_value, color='g', linestyle='-', 
                       label=f'True: {true_value:.6f}')
    
    # Compute bootstrap confidence interval
    ci = np.percentile(bootstrap_params[name], [2.5, 97.5])
    axes[i].axvline(ci[0], color='b', linestyle=':', label=f'2.5%: {ci[0]:.6f}')
    axes[i].axvline(ci[1], color='b', linestyle=':', label=f'97.5%: {ci[1]:.6f}')
    
    axes[i].set_title(f'Bootstrap Distribution of {name}')
    axes[i].set_xlabel(f'{name}')
    axes[i].set_ylabel('Density')
    axes[i].legend()
    axes[i].grid(True)

plt.tight_layout()
plt.show()

# Print bootstrap confidence intervals
print("95% Bootstrap Confidence Intervals:")
for name in param_names:
    ci = np.percentile(bootstrap_params[name], [2.5, 97.5])
    print(f"{name}: [{ci[0]:.6f}, {ci[1]:.6f}]")

### Residual Bootstrap for GARCH Models

Now let's use residual bootstrap as an alternative approach. We'll resample the standardized residuals from the estimated model, generate new returns series, and re-estimate the model on each bootstrap sample.

In [ ]:
# Residual bootstrap for GARCH models
def residual_bootstrap_garch(returns: np.ndarray, model: GARCH, 
                            estimated_params: np.ndarray, 
                            std_residuals: np.ndarray,
                            num_bootstrap: int = 1000) -> Dict[str, np.ndarray]:
    """
    Perform residual bootstrap for GARCH models.
    
    Parameters
    ----------
    returns : np.ndarray
        Original returns data
    model : GARCH
        Estimated GARCH model
    estimated_params : np.ndarray
        Estimated parameters
    std_residuals : np.ndarray
        Standardized residuals from the estimated model
    num_bootstrap : int, optional
        Number of bootstrap replications
        
    Returns
    -------
    Dict[str, np.ndarray]
        Dictionary of bootstrap parameter distributions
    """
    # Get parameter names
    param_names = model.parameter_names
    
    # Initialize arrays to store bootstrap parameters
    bootstrap_params = {name: np.zeros(num_bootstrap) for name in param_names}
    
    # Extract GARCH parameters
    omega = estimated_params[0]
    alpha = estimated_params[1]
    beta = estimated_params[2]
    
    # Number of observations
    n = len(returns)
    
    # Perform bootstrap replications
    for i in range(num_bootstrap):
        # Progress update
        if (i + 1) % 100 == 0 or i == 0:
            print(f"Bootstrap replication {i + 1}/{num_bootstrap}")
        
        # Resample standardized residuals
        bootstrap_indices = np.random.choice(n, size=n, replace=True)
        bootstrap_residuals = std_residuals[bootstrap_indices]
        
        # Generate bootstrap returns
        bootstrap_returns = np.zeros(n)
        bootstrap_sigma2 = np.zeros(n)
        bootstrap_sigma2[0] = np.var(returns)  # Initial variance
        
        for t in range(1, n):
            # GARCH variance
            bootstrap_sigma2[t] = omega + alpha * bootstrap_returns[t-1]**2 + beta * bootstrap_sigma2[t-1]
            
            # Returns
            bootstrap_returns[t] = np.sqrt(bootstrap_sigma2[t]) * bootstrap_residuals[t]
        
        try:
            # Re-estimate the model on bootstrap data
            bootstrap_result = model.fit(bootstrap_returns)
            
            # Store parameters
            for j, name in enumerate(param_names):
                bootstrap_params[name][i] = bootstrap_result.parameters[j]
        except Exception as e:
            # If estimation fails, use original parameters
            print(f"Warning: Estimation failed for bootstrap sample {i + 1}: {e}")
            for j, name in enumerate(param_names):
                bootstrap_params[name][i] = estimated_params[j]
    
    return bootstrap_params

# Get standardized residuals from the estimated model
std_residuals = garch_results.standardized_residuals

# Run residual bootstrap (with fewer replications for demonstration)
num_bootstrap = 200  # Use a smaller number for demonstration
residual_bootstrap_params = residual_bootstrap_garch(
    returns_data, garch_model, garch_results.parameters, std_residuals, num_bootstrap=num_bootstrap
)

In [ ]:
# Compare parametric and residual bootstrap distributions
param_names = garch_model.parameter_names
fig, axes = plt.subplots(len(param_names), 1, figsize=(14, 4 * len(param_names)))

for i, name in enumerate(param_names):
    # Plot bootstrap distributions
    sns.kdeplot(bootstrap_params[name], ax=axes[i], label='Parametric Bootstrap')
    sns.kdeplot(residual_bootstrap_params[name], ax=axes[i], label='Residual Bootstrap')
    
    # Add vertical line for estimated value
    axes[i].axvline(garch_results.parameters[i], color='r', linestyle='--', 
                   label=f'Estimated: {garch_results.parameters[i]:.6f}')
    
    # Add vertical line for true value if available
    true_value = None
    if name == 'omega':
        true_value = true_omega
    elif name == 'alpha[1]':
        true_value = true_alpha
    elif name == 'beta[1]':
        true_value = true_beta
    
    if true_value is not None:
        axes[i].axvline(true_value, color='g', linestyle='-', 
                       label=f'True: {true_value:.6f}')
    
    axes[i].set_title(f'Bootstrap Distributions of {name}')
    axes[i].set_xlabel(f'{name}')
    axes[i].set_ylabel('Density')
    axes[i].legend()
    axes[i].grid(True)

plt.tight_layout()
plt.show()

# Print and compare bootstrap confidence intervals
print("95% Bootstrap Confidence Intervals:")
for name in param_names:
    param_ci = np.percentile(bootstrap_params[name], [2.5, 97.5])
    resid_ci = np.percentile(residual_bootstrap_params[name], [2.5, 97.5])
    print(f"{name}:")
    print(f"  Parametric: [{param_ci[0]:.6f}, {param_ci[1]:.6f}], Width = {param_ci[1] - param_ci[0]:.6f}")
    print(f"  Residual:   [{resid_ci[0]:.6f}, {resid_ci[1]:.6f}], Width = {resid_ci[1] - resid_ci[0]:.6f}")

## 5. Model Confidence Set (MCS)

The Model Confidence Set (MCS) procedure identifies the set of models that are statistically indistinguishable from the best model based on a user-defined loss function.

In [ ]:
# Generate loss data for 5 different models over 100 time periods
np.random.seed(42)
n_models = 5
n_periods = 100

# Model 0 and 1 are the best, others are worse
base_losses = np.random.normal(0, 1, (n_periods, n_models))
base_losses[:, 2:] += 0.5  # Models 2-4 have higher loss

# Create model names
model_names = [f"Model {i+1}" for i in range(n_models)]

# Create a Model Confidence Set
mcs = ModelConfidenceSet(
    block_size=10,           # Block size for bootstrap
    num_bootstrap=1000,      # Number of bootstrap replications
    significance_level=0.05  # Significance level
)

# Run the MCS procedure
mcs_result = mcs.run(base_losses, model_names=model_names)

# Print results
print("Model Confidence Set Results:")
print(f"Included models: {[model_names[i] for i in mcs_result.included_models]}")
print(f"Excluded models: {[model_names[i] for i in mcs_result.excluded_models]}")
print("
Model p-values:")
for i, p_val in enumerate(mcs_result.pvalues):
    print(f"{model_names[i]}: {p_val:.4f}")

In [ ]:
# Visualize MCS results
plt.figure(figsize=(12, 6))
bars = plt.bar(model_names, mcs_result.pvalues)

# Color bars based on inclusion in MCS
for i, model_idx in enumerate(range(n_models)):
    if model_idx in mcs_result.included_models:
        bars[i].set_color('green')
    else:
        bars[i].set_color('red')

plt.axhline(mcs.significance_level, color='black', linestyle='--', 
            label=f'Significance Level ({mcs.significance_level})')
plt.title('Model Confidence Set p-values')
plt.ylabel('p-value')
plt.xlabel('Model')
plt.legend()
plt.grid(True, axis='y')
plt.tight_layout()
plt.show()

### MCS with Different Bootstrap Methods

Let's compare the results of the MCS procedure using different bootstrap methods.

In [ ]:
# Run MCS with block bootstrap
mcs_block = ModelConfidenceSet(
    block_size=10,
    num_bootstrap=1000,
    significance_level=0.05,
    bootstrap_method='block'  # Explicitly specify block bootstrap
)
mcs_block_result = mcs_block.run(base_losses, model_names=model_names)

# Run MCS with stationary bootstrap
mcs_stationary = ModelConfidenceSet(
    block_size=10,  # Used as expected block size for stationary bootstrap
    num_bootstrap=1000,
    significance_level=0.05,
    bootstrap_method='stationary'  # Specify stationary bootstrap
)
mcs_stationary_result = mcs_stationary.run(base_losses, model_names=model_names)

# Compare results
print("MCS Results with Block Bootstrap:")
print(f"Included models: {[model_names[i] for i in mcs_block_result.included_models]}")
print("
MCS Results with Stationary Bootstrap:")
print(f"Included models: {[model_names[i] for i in mcs_stationary_result.included_models]}")

# Compare p-values
print("
Model p-values:")
for i in range(n_models):
    print(f"{model_names[i]}: Block = {mcs_block_result.pvalues[i]:.4f}, Stationary = {mcs_stationary_result.pvalues[i]:.4f}")

In [ ]:
# Visualize comparison of p-values
plt.figure(figsize=(12, 6))

x = np.arange(len(model_names))
width = 0.35

plt.bar(x - width/2, mcs_block_result.pvalues, width, label='Block Bootstrap')
plt.bar(x + width/2, mcs_stationary_result.pvalues, width, label='Stationary Bootstrap')

plt.axhline(0.05, color='black', linestyle='--', label='Significance Level (0.05)')
plt.xlabel('Model')
plt.ylabel('p-value')
plt.title('Comparison of MCS p-values with Different Bootstrap Methods')
plt.xticks(x, model_names)
plt.legend()
plt.grid(True, axis='y')
plt.tight_layout()
plt.show()

## 6. Bootstrap Reality Check and SPA Test

The Bootstrap Reality Check (BRC) and Superior Predictive Ability (SPA) tests evaluate whether any model in a set outperforms a benchmark model.

In [ ]:
# Generate loss data for benchmark and 5 competing models
np.random.seed(42)
n_periods = 100
n_models = 5

# Benchmark model losses
benchmark_losses = np.random.normal(0, 1, n_periods)

# Competing models' losses (model 0 is better, others are not)
model_losses = np.random.normal(0, 1, (n_periods, n_models))
model_losses[:, 0] -= 0.3  # Model 0 has lower loss

# Create model names
model_names = [f"Model {i+1}" for i in range(n_models)]

# Create a BSDS test
bsds = BSDS(
    block_size=10,           # Block size for bootstrap
    num_bootstrap=1000,      # Number of bootstrap replications
    seed=42                  # Random seed for reproducibility
)

# Run the test
bsds_result = bsds.run(benchmark_losses, model_losses, model_names=model_names)

# Print results
print("BSDS Test Results:")
print(f"Reality Check p-value: {bsds_result.rc_pvalue:.4f}")
print(f"SPA p-value: {bsds_result.spa_pvalue:.4f}")
print("
Individual model p-values:")
for i, p_val in enumerate(bsds_result.model_pvalues):
    print(f"{model_names[i]}: {p_val:.4f}")

In [ ]:
# Visualize BSDS results
# Calculate loss differences
loss_diffs = np.mean(benchmark_losses.reshape(-1, 1) - model_losses, axis=0)

plt.figure(figsize=(12, 6))
bars = plt.bar(model_names, loss_diffs)

# Color bars based on significance
for i, p_val in enumerate(bsds_result.model_pvalues):
    if p_val < 0.05:
        bars[i].set_color('green')
    else:
        bars[i].set_color('red')

plt.axhline(0, color='black', linestyle='--', label='Benchmark')
plt.title('Average Loss Difference vs Benchmark')
plt.ylabel('Benchmark Loss - Model Loss')
plt.xlabel('Model')
plt.legend()
plt.grid(True, axis='y')
plt.tight_layout()
plt.show()

# Plot p-values
plt.figure(figsize=(12, 6))
plt.bar(model_names, bsds_result.model_pvalues)
plt.axhline(0.05, color='red', linestyle='--', label='Significance Level (0.05)')
plt.title('Individual Model p-values')
plt.ylabel('p-value')
plt.xlabel('Model')
plt.legend()
plt.grid(True, axis='y')
plt.tight_layout()
plt.show()

## 7. Custom Bootstrap Functions

The MFE Toolbox allows you to create custom bootstrap functions for specialized applications. Let's implement a bootstrap function for Sharpe ratio confidence intervals.

In [ ]:
def bootstrap_sharpe_ratio(returns: np.ndarray, block_size: int = 20, 
                          num_samples: int = 1000) -> Tuple[float, float, float]:
    """
    Compute bootstrap confidence interval for Sharpe ratio.
    
    Parameters
    ----------
    returns : np.ndarray
        Array of return data
    block_size : int, optional
        Size of bootstrap blocks
    num_samples : int, optional
        Number of bootstrap samples
        
    Returns
    -------
    Tuple[float, float, float]
        Tuple containing (sharpe_ratio, lower_bound, upper_bound)
    """
    # Calculate sample Sharpe ratio
    sample_sharpe = returns.mean() / returns.std()
    
    # Create bootstrap
    bootstrap = BlockBootstrap(block_size=block_size)
    bootstrap_samples = bootstrap.generate(returns, num_samples=num_samples)
    
    # Compute Sharpe ratio for each bootstrap sample
    bootstrap_sharpes = np.array([
        sample.mean() / sample.std() for sample in bootstrap_samples
    ])
    
    # Compute confidence interval
    conf_interval = np.percentile(bootstrap_sharpes, [2.5, 97.5])
    
    return sample_sharpe, conf_interval[0], conf_interval[1]

# Generate some return data
np.random.seed(42)
returns = np.random.normal(0.001, 0.01, 1000)  # Daily returns with 0.1% mean and 1% volatility

# Compute Sharpe ratio and confidence interval
sharpe, lower, upper = bootstrap_sharpe_ratio(returns)
print(f"Sharpe Ratio: {sharpe:.4f}")
print(f"95% Confidence Interval: [{lower:.4f}, {upper:.4f}]")

# Plot bootstrap distribution of Sharpe ratio
bootstrap = BlockBootstrap(block_size=20)
bootstrap_samples = bootstrap.generate(returns, num_samples=1000)
bootstrap_sharpes = np.array([sample.mean() / sample.std() for sample in bootstrap_samples])

plt.figure(figsize=(14, 6))
sns.histplot(bootstrap_sharpes, kde=True, bins=50)
plt.axvline(sharpe, color='r', linestyle='--', label=f'Sample Sharpe: {sharpe:.4f}')
plt.axvline(lower, color='b', linestyle=':', label=f'2.5%: {lower:.4f}')
plt.axvline(upper, color='b', linestyle=':', label=f'97.5%: {upper:.4f}')
plt.title('Bootstrap Distribution of Sharpe Ratio')
plt.xlabel('Sharpe Ratio')
plt.ylabel('Frequency')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

## 8. Performance Considerations

The bootstrap methods in the MFE Toolbox are optimized for performance using Numba's just-in-time compilation. Here are some considerations for optimal performance:

In [ ]:
# Measure performance for different data sizes
data_sizes = [1000, 5000, 10000]
block_size = 50
num_samples = 100

print("Performance for Different Data Sizes:")
for size in data_sizes:
    # Generate data
    data = np.random.normal(0, 1, size)
    
    # Time Numba-accelerated version
    bootstrap = BlockBootstrap(block_size=block_size)
    start_time = time.time()
    _ = bootstrap.generate(data, num_samples=num_samples)
    numba_time = time.time() - start_time
    
    print(f"Data size = {size}: {numba_time:.4f} seconds for {num_samples} bootstrap samples")

# Measure performance for different numbers of bootstrap samples
data_size = 1000
block_size = 50
sample_sizes = [100, 500, 1000]

print("
Performance for Different Numbers of Bootstrap Samples:")
for num_samples in sample_sizes:
    # Generate data
    data = np.random.normal(0, 1, data_size)
    
    # Time Numba-accelerated version
    bootstrap = BlockBootstrap(block_size=block_size)
    start_time = time.time()
    _ = bootstrap.generate(data, num_samples=num_samples)
    numba_time = time.time() - start_time
    
    print(f"Number of samples = {num_samples}: {numba_time:.4f} seconds for data size {data_size}")

### Asynchronous Processing for Large Datasets

For large datasets or many bootstrap samples, asynchronous processing can significantly improve responsiveness.

In [ ]:
# Define an asynchronous function to run bootstrap with progress tracking
async def run_large_bootstrap_async() -> np.ndarray:
    """
    Run bootstrap on a large dataset asynchronously with progress tracking.
    
    Returns
    -------
    np.ndarray
        Bootstrap statistics
    """
    # Generate a large dataset
    np.random.seed(42)
    data_size = 10000
    data = np.random.normal(0, 1, data_size)
    
    # Create a bootstrap object
    bootstrap = BlockBootstrap(block_size=50)
    
    # Define a progress callback function
    def progress_callback(percent: float, message: str) -> None:
        print(f"{percent:.1f}% complete: {message}")
    
    # Generate bootstrap samples asynchronously with progress tracking
    print(f"Generating 1,000 bootstrap samples for data size {data_size}...")
    bootstrap_samples = await bootstrap.generate_async(
        data, 
        num_samples=1000,
        progress_callback=progress_callback
    )
    
    # Compute bootstrap statistics
    bootstrap_means = np.array([sample.mean() for sample in bootstrap_samples])
    
    return bootstrap_means

# Run the async function
bootstrap_means = await run_large_bootstrap_async()

# Plot the bootstrap distribution
plt.figure(figsize=(14, 6))
sns.histplot(bootstrap_means, kde=True, bins=50)
plt.axvline(bootstrap_means.mean(), color='r', linestyle='--', label=f'Mean: {bootstrap_means.mean():.4f}')
plt.title('Bootstrap Distribution of Mean for Large Dataset')
plt.xlabel('Mean')
plt.ylabel('Frequency')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

## 9. Conclusion

In this notebook, we've explored the bootstrap methods available in the MFE Toolbox for financial time series analysis. We've covered:

1. **Block Bootstrap**: Resampling blocks of consecutive observations to preserve short-range dependence
2. **Stationary Bootstrap**: Using random block lengths for improved stationarity properties
3. **Bootstrap for Parameter Uncertainty**: Assessing uncertainty in GARCH model parameters
4. **Model Confidence Set (MCS)**: Identifying the set of models that are statistically indistinguishable from the best model
5. **Bootstrap Reality Check and SPA Test**: Testing for superior predictive ability among competing forecasting models
6. **Custom Bootstrap Functions**: Creating specialized bootstrap applications
7. **Performance Considerations**: Optimizing bootstrap methods for large datasets

The bootstrap methods in the MFE Toolbox provide powerful tools for statistical inference with dependent data. By leveraging NumPy's efficient array operations and Numba's performance acceleration, these methods enable robust analysis of financial time series data.

The implementation in Python with Numba acceleration offers significant advantages over the previous MATLAB implementation, including improved performance, better integration with the Python ecosystem, and support for asynchronous processing with progress tracking.

In [ ]:
# Display version information
print(f"MFE Toolbox version: {mfe.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")
print(f"Matplotlib version: {plt.matplotlib.__version__}")
print(f"Seaborn version: {sns.__version__}")